In [91]:
!pip install google-adk -qq

In [92]:
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [93]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


# MCP Agent For Image Generation

In [94]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

from IPython.display import Image, display
import json
import asyncio

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [95]:
# MCP 
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@pollinations/model-context-protocol",
            ],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")


✅ MCP Tool created


In [96]:
instruction_text = """
You are an image-generation agent. Follow these rules exactly:

1) URL construction:
   - Always build image URLs using this exact template:
     https://image.pollinations.ai/prompt/[PERCENT_ENCODED_PROMPT]
   - Use strict percent-encoding:
       space → %20
       ? → %3F
       ! → %21
       etc.
   - Do not remove or change any user words or punctuation.

2) Tool usage:
   - Always call the MCP image tool with:
       {'prompt': '<original user prompt>'}
   - If the tool returns a JSON with an imageUrl field, return it directly.
   - If the tool fails or returns truncated text, regenerate the URL yourself using rule (1).

3) Output format:
   - Always return a JSON object:
       {"imageUrl": "<full_percent_encoded_url>"}

4) Safety:
   - If the prompt is disallowed (illegal, harmful, sexual content with minors), return:
       {"error": "refused"}

5) Examples:
   User: "What is artificial intelligence?"
   → Percent-encoded prompt: "What%20is%20artificial%20intelligence%3F"
   → URL:
     https://image.pollinations.ai/prompt/What%20is%20artificial%20intelligence%3F
   → Return:
     {"imageUrl": "https://image.pollinations.ai/prompt/What%20is%20artificial%20intelligence%3F"}

   User: "pink pony girl"
   → "pink%20pony%20girl"
   → {"imageUrl": "https://image.pollinations.ai/prompt/pink%20pony%20girl"}
"""


In [97]:
# Create image agent with MCP integration
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction=instruction_text,
    tools=[mcp_image_server],
)

In [98]:
from google.adk.runners import InMemoryRunner
import json

runner = InMemoryRunner(agent=image_agent)
response = await runner.run_debug("Provide a image of spider man", verbose=True)

url = json.loads(response[2].content.parts[0].text)["imageUrl"]
print(url)
display(Image(url=url))
print('Image Displayed Successfully')   


 ### Created new session: debug_session_id

User > Provide a image of spider man


/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


image_agent > [Calling tool: generateImageUrl({'prompt': 'spider man'})]
image_agent > [Tool result: {'content': [{'type': 'text', 'text': '{\n  "imageUrl": "https://image.pollinations.ai/prompt/spider...]


TypeError: 'NoneType' object is not subscriptable

# Long-Running Operations for Human Approval

In [132]:
## LOR Function

MAX_IMG_APPROVE = 2

def max_image_approval(num_img: int, tool_context: ToolContext):
    """Generating Image Requires approval if generate more than 2 containers (MAX_IMG_APPROVE).

    Args:
        num_img: Number of image to generate

    Returns:
        Dictionary with image generation status
    """
    # SCENARIO 1: Small orders 
    if num_img <= MAX_IMG_APPROVE:
        return {
            "status": "approved",
            "image_id": f"ORD-{num_img}-AUTO",
            "num_img": num_img,
            "message": f"Image auto-approved: {num_img}",
        }

    # SCENARIO 2: This is the first time this tool is called. Large orders need human approval - PAUSE here.
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"Bulk Image order: {num_img}. Do you want to approve?",
            payload={"num_img": num_img,},
        )
        return {  # This is sent to the Agent
            "status": "pending",
            "message": f"Order for {num_img} containers requires approval",
        }

    # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_img}-HUMAN",
            "num_img": num_img,
            "message": f"Image approved: {num_img}",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Bulk Image rejected: {num_img}",
        }
print("✅ Long-running functions created!")

✅ Long-running functions created!


In [133]:
## LOR Agent
image_agent = LlmAgent(
    name="image_agent",
    model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction="""
    You are a dedicated image-generation agent called only when an image is needed.
    
    Rules:
    
    1. Treat EVERY input as a request to generate images.
    2. First, figure out how many images the user wants. 
       - If the message contains a number N (e.g. "4 images"), use that.
       - If not clear, default to 1.
    3. Call the `max_image_approval` tool with:
           {"num_img": N}
    4. If the tool response status is "rejected", reply ONLY with:
           {"error": "<short_reason>"}
       and STOP.
    5. If approved, generate images using the Pollinations MCP tool:
       - If the tool supports multiple images in one call (e.g. a `count` or `batch_size` parameter), use N.
       - Otherwise, call it N times.
    6. Build your final response as:
       - If N == 1:
             {"imageUrl": "<single_url>"}
       - If N > 1:
             {"imageUrls": ["<url1>", "<url2>", ...]}
    7. Return Image in form of a dictionary
    8. Do NOT output anything except that JSON. No markdown, no text, no backticks.
    9. If the Output is None Say : "Sorry Not Able to Process Image Right Now Run Again"
    """,

    tools=[FunctionTool(func=max_image_approval), mcp_image_server],
)

print("LOR Agent Created")


LOR Agent Created


In [134]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS!
image_app = App(
    name="Image_coordinator",
    root_agent=image_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app created!")

✅ Resumable app created!


/tmp/ipykernel_48/3014372566.py:5: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [135]:
session_service = InMemorySessionService()

# Create runner with the resumable app
Image_runner = Runner(
    app=image_app,  # Pass the app instead of the agent
    session_service=session_service,
)

print("✅ Runner created!")

✅ Runner created!


# WorkFlow for Image Generation (LRO and MCP)

In [136]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


print("✅ Helper functions defined")

✅ Helper functions defined


In [137]:
async def run_image_workflow(query: str):
    """Runs an image workflow with approval handling.

    Args:
        query: User's image request
    """
    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"

    # Create session
    await session_service.create_session(
        app_name=image_app.name,
        user_id="test_user",
        session_id=session_id,
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

    # STEP 1: Send initial request to the agent
    async for event in Image_runner.run_async(
        user_id="test_user",
        session_id=session_id,
        new_message=query_content,
    ):
        events.append(event)

    # Helper: extract last text part from a list of events
    def extract_final_image_json(events_list):
        """
        Returns a JSON string like:
          '{"imageUrl": "..."}'
        by checking:
          1) final LLM text
          2) tool function_response with imageUrl / urls
        """
        last_text = None
        last_image_url = None
        last_image_urls = None  # for multi-image extensions
    
        for ev in events_list:
            if getattr(ev, "content", None) and ev.content.parts:
                for part in ev.content.parts:
                    # 1) Normal text from the agent
                    if getattr(part, "text", None):
                        last_text = part.text
    
                    # 2) Tool responses (MCP / function tools)
                    if getattr(part, "function_response", None):
                        resp = part.function_response.response
                        if isinstance(resp, dict):
                            if "imageUrl" in resp:
                                last_image_url = resp["imageUrl"]
                            elif "urls" in resp and isinstance(resp["urls"], list):
                                last_image_urls = resp["urls"]
    
        # Prefer explicit LLM JSON text if it exists
        if last_text:
            return last_text
    
        # Fallback: tool returned a single imageUrl
        if last_image_url:
            return json.dumps({"imageUrl": last_image_url})
    
        # Fallback: tool returned multiple urls (if your MCP does that)
        if last_image_urls:
            return json.dumps({"imageUrls": last_image_urls})
    
        return None
    
    
        # STEP 2: Check collected events for approval request
    approval_info = check_for_approval(events)

    # This will hold the final agent JSON string
    final_response_text = None

    # STEP 3: If approval is required, pause and ask human
    if approval_info:
        print("⏸️  Agent requested approval. Human intervention required.")

        def prompt_for_decision():
            while True:
                resp = input("Approve? (y/N): ").strip().lower()
                if resp in ("y", "yes"):
                    return True
                if resp in ("n", "no", ""):
                    return False
                print("Please type 'y' or 'n'.")

        decision = await asyncio.get_event_loop().run_in_executor(
            None, prompt_for_decision
        )
        print(f"🤔 Human Decision: {'APPROVE ✅' if decision else 'REJECT ❌'}\n")

        # Resume the agent with the human decision
        approval_events = []
        async for event in Image_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(approval_info, decision),
            invocation_id=approval_info["invocation_id"],
        ):
            approval_events.append(event)

            # Still print streaming text for debugging, like you did
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if getattr(part, "text", None):
                        print(f"Agent > {part.text}")

        # Get the final JSON string from the approval run
        final_response_text = extract_final_image_json(approval_events)

    else:
        # No approval required
        print_agent_response(events)
        final_response_text = extract_final_image_json(events)

    print(f"{'='*60}\n")

    # 🔴 THIS IS THE IMPORTANT PART
    return final_response_text


In [140]:
res = await run_image_workflow("Generate 4 image of iron man")
print(res)
data = json.loads(res)

if "imageUrls" in data:
    for u in data["imageUrls"]:
        display(Image(url=u))
elif "imageUrl" in data:
    display(Image(url=data["imageUrl"]))
else:
    print("Error:", data.get("error"))


# IF Gives None Type Error Please Run the cell again


User > Generate 4 image of iron man



⏸️  Agent requested approval. Human intervention required.


Approve? (y/N):  y


🤔 Human Decision: APPROVE ✅



Agent > {"imageUrls": ["https://image.pollinations.ai/prompt/iron%20man?width=1024&height=1024", "https://image.pollinations.ai/prompt/iron%20man?width=1024&height=1024", "https://image.pollinations.ai/prompt/iron%20man?width=1024&height=1024", "https://image.pollinations.ai/prompt/iron%20man?width=1024&height=1024"]}

{"imageUrls": ["https://image.pollinations.ai/prompt/iron%20man?width=1024&height=1024", "https://image.pollinations.ai/prompt/iron%20man?width=1024&height=1024", "https://image.pollinations.ai/prompt/iron%20man?width=1024&height=1024", "https://image.pollinations.ai/prompt/iron%20man?width=1024&height=1024"]}
